# Take in raw fluorescence images and compute a scalar fluorescence score

In [1]:
%load_ext autoreload
%autoreload 2

import zarr
import dask.array as da
import pandas as pd
import numpy as np
from scipy import stats
from scipy import ndimage as ndi

from pathlib import Path
from bsccm import BSCCM 


home = str(Path.home())
data_root = home + '/BSCCM_local/BSCCM/'

dataset = BSCCM(data_root)


In [ ]:
fluor_dataframe = pd.DataFrame()
# copy over global insex
fluor_dataframe.insert(0, 'global_index', dataset.index_dataframe.global_index.to_list())

for index in dataset.index_dataframe.global_index:
    index = int(index)
    print('{}          \r'.format(index), end='')

    for ch_index, fluor_channel in enumerate(dataset.global_metadata['fluorescence']['channel_names']):
        #### get shading correction ####
#         background = dataset.get_unstained_background(index, fluor_channel).astype(np.float32)
#         foreground = dataset.get_background(index, contrast_type='fluor', 
#                                             channel=fluor_channel, percentile=50).astype(np.float32)
#         shading = foreground - background
#         # smooth
#         shading = ndi.gaussian_filter(shading, 2.5)
        
#         if np.any(shading <= 0):
#             num = np.sum(shading <= 0)
#             if num > 50:
#                 print('{} nonpositive shading pix detected at index {}   ch_index {}'.format(num, index, ch_index))
#             #hot pixels seem to creat occasional 0 or subzero patches
#             shading[shading <= 0] = np.min(shading[shading > 0])
     
#         # normalize shaing correction so that it doesn't attenuate absolute fluorescence value
#         shading = shading / np.linalg.norm(shading)
    
        #compute fluorescence
        image = dataset.read_image(index, contrast_type='fluor', channel=fluor_channel).astype(np.float32)
#         background_subtracted = image.astype(np.float32) - background
#         corrected = background_subtracted / shading

        yy, xx = np.meshgrid(np.arange(image.shape[0]), np.arange(image.shape[0]))
        fluor_mask = np.sqrt((yy - image.shape[0]/2) ** 2 + (xx - image.shape[1]/2) ** 2) < image.shape[0]/2
        fluor_mask_pixels = image[fluor_mask]
        fluor_background_pixels = image[np.logical_not(fluor_mask)]
        
        # clip at zero here before summing so you dont get everything at exactly equal to 0
#         fluor_mask_pixels[fluor_mask_pixels < 0] = 0
        total_fluor = np.sum(fluor_mask_pixels)
        fluor_dataframe.loc[index, fluor_channel + '_total_raw'] = total_fluor
        total_background = np.sum(fluor_background_pixels)
        fluor_dataframe.loc[index, fluor_channel + '_background'] = total_background
    
        #same thing but with local background subtraction
#         local_background = np.percentile(fluor_mask_pixels, 5)
#         total_fluor_local_background_subtraction = np.sum(fluor_mask_pixels - local_background)
#         fluor_dataframe.loc[index, fluor_channel + '_total_local_background_subtracted'] = total_fluor
    
        
    
#         local_background = np.percentile(fluor_mask_pixels, 5, axis=1)
#         mean_fluor = np.sum(fluor_mask_pixels - local_background[:, None], axis=1)
#         mean_fluor_without_local_subtraction = np.sum(fluor_mask_pixels, axis=1)


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
fluor_dataframe.to_csv(data_root + 'BSCCM_surface_markers.csv', index=False)

Experimenting with how to get rid of values less than 0 in computed shading...seems like gaussian filtering the image supresses these noisy values and solves problem

In [ ]:



background_crop = crop(background, (y, x))
        if shading is not None:
            shading_crop = crop(shading, (y, x))
        else:
            shading_crop = np.ones_like(background_crop)
        fluor_crop = crops_dataset['{}/{}'.format(dataset_index, crop_name)].get_orthogonal_selection(
            (fluor_channel_indices, slice(None), slice(None)))
        fluor_crop_corrected = (fluor_crop - background_crop) / shading_crop
        # mask only pixels within 130 pixel diameter circle
        yy, xx = np.meshgrid(np.arange(150), np.arange(150))
        fluor_mask = np.sqrt((yy - 75) ** 2 + (xx - 75) ** 2) < 65
        fluor_mask_pixels = fluor_crop_corrected[:, fluor_mask]
        local_background = np.percentile(fluor_mask_pixels, 5, axis=1)
        mean_fluor = np.sum(fluor_mask_pixels - local_background[:, None], axis=1)
        mean_fluor_without_local_subtraction = np.sum(fluor_mask_pixels, axis=1)

        # napari_show({'b': background_crop, 's': shading_crop, 'f': fluor_crop, 'f-b': fluor_crop - background_crop,
        #              'corrected': fluor_crop_corrected})

        mean_fluor[mean_fluor < 0] = 0
        mean_fluor_without_local_subtraction[mean_fluor_without_local_subtraction < 0]
        mean_fluors.append(mean_fluor)
        mean_fluors_without_background_sub.append(mean_fluor_without_local_subtraction)

In [ ]:


# Which dataset indices correspond to which flatfield
# TODO Replace this with something read from the dataframe
# TODO Need to account for dataset 13 being off...

fluor_channel_indices = np.array([crops_dataset[str(datasets_920[0])]['all_blobs'].attrs['channel_names'].index(n) for n in
                   ['F1_BV711', 'F2_BV650', 'F3_BV605', 'F4_BV570', 'F5_BV510', 'F6_BV421']])

fluor_names = ['BV711', 'BV650', 'BV605', 'BV570', 'BV510', 'BV421']
new_cols = {name: np.zeros((dataframe.size, )) for name in fluor_names + [n + '_without_local_background_sub' for n in fluor_names]}
#add new columns to dataframe
dataframe = dataframe.join(pd.DataFrame(new_cols))


def compute_fluoresence(dataframe, dataset_index, background, shading=None):
    """
    Do background substraction and optionally, shading
    """
    print('dataset {}'.format(dataset_index))
    in_dataset_indices = np.flatnonzero(dataframe['dataset_index'] == dataset_index)
    dataset_dataframe = dataframe.loc[in_dataset_indices]
    y_coords = dataset_dataframe['blob_y'].to_numpy()
    x_coords = dataset_dataframe['blob_x'].to_numpy()
    crop_names = dataset_dataframe['blob_name'].to_list()

    mean_fluors_without_background_sub = []
    mean_fluors = []
    for i, (y, x, crop_name) in enumerate(zip(y_coords, x_coords, crop_names)):
        print('{} of {}\r'.format(i, y_coords.size), end='')
        background_crop = crop(background, (y, x))
        if shading is not None:
            shading_crop = crop(shading, (y, x))
        else:
            shading_crop = np.ones_like(background_crop)
        fluor_crop = crops_dataset['{}/{}'.format(dataset_index, crop_name)].get_orthogonal_selection(
            (fluor_channel_indices, slice(None), slice(None)))
        fluor_crop_corrected = (fluor_crop - background_crop) / shading_crop
        # mask only pixels within 130 pixel diameter circle
        yy, xx = np.meshgrid(np.arange(150), np.arange(150))
        fluor_mask = np.sqrt((yy - 75) ** 2 + (xx - 75) ** 2) < 65
        fluor_mask_pixels = fluor_crop_corrected[:, fluor_mask]
        local_background = np.percentile(fluor_mask_pixels, 5, axis=1)
        mean_fluor = np.sum(fluor_mask_pixels - local_background[:, None], axis=1)
        mean_fluor_without_local_subtraction = np.sum(fluor_mask_pixels, axis=1)

        # napari_show({'b': background_crop, 's': shading_crop, 'f': fluor_crop, 'f-b': fluor_crop - background_crop,
        #              'corrected': fluor_crop_corrected})

        mean_fluor[mean_fluor < 0] = 0
        mean_fluor_without_local_subtraction[mean_fluor_without_local_subtraction < 0]
        mean_fluors.append(mean_fluor)
        mean_fluors_without_background_sub.append(mean_fluor_without_local_subtraction)

    without_local_sub = np.stack(mean_fluors_without_background_sub)
    mean_fluors = np.stack(mean_fluors)
    return mean_fluors, without_local_sub

def correct_and_compute_fluorescence(ids, do_local_backgrounds=True, shading=True):
    # Compute shading corrections from loess of all stain data
    mean_fluorescences = {}
    without_background_subtraction = {}
    for dataset in ids:
        if dataset in datasets_813:
            background = shading_file['813_backgrounds']
            ff = shading_file['813_ff_slide']
        elif dataset in datasets_828:
            background = shading_file['828_backgrounds']
            ff = shading_file['828_ff_slide']
        elif dataset in datasets_920:
            background = shading_file['920_backgrounds']
            ff = shading_file['920_ff_slide']
        shading_corr = ff - 0.92 * background

        #normalize by median brightness
        shading_corr = shading_corr / np.mean(shading_corr, axis=(1,2))[:,None,None]

        mean_fluorescence, fluor_without_local_background = compute_fluoresence(dataframe, dataset, background,
                                    shading=shading_corr if shading else None)
        mean_fluorescences[dataset] = mean_fluorescence
        without_background_subtraction[dataset] = fluor_without_local_background
    return mean_fluorescences, without_background_subtraction


ids = np.unique(dataframe['dataset_index'].to_numpy().astype(np.int))

mean_fluorescences, mean_fluorescences_no_bckd_sub = correct_and_compute_fluorescence(ids, do_local_backgrounds=True, shading=True)
for dataset_index in ids:
    in_dataset_indices = np.flatnonzero(dataframe['dataset_index'] == dataset_index)
    new_data = np.concatenate([mean_fluorescences[dataset_index],
                               mean_fluorescences_no_bckd_sub[dataset_index]], axis=1)
    for i, col_name in enumerate(new_cols.keys()):
        dataframe.loc[in_dataset_indices, col_name] = new_data[:, i]


dataframe.to_csv(data_root + 'crops_filtered_record_single_with_fluor_normalized_ff.csv', index=False)
